In [1]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [2]:
file_path = "/Users/nainishdhanorkar/Downloads/task/macbook-air-13inch-m4-2025-info (1).pdf"

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [3]:
loader = PyPDFLoader(file_path=file_path)
documents = loader.load()
len(documents)

2

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=120,
)

splits = text_splitter.split_documents(documents)
len(splits), splits[0].page_content[:200]

(9,
 'Before using MacBook Air, review the MacBook Air  \nGetting Started Guide  at support.apple.com/guide/\nmacbook-air. Retain documentation for future \nreference.\nSafety and Handling\nSee “Safety, handling')

In [5]:
vectorstore = FAISS.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [6]:
prompt = ChatPromptTemplate.from_template(
    "You are a concise assistant. Use the context to answer.\n"
    "If the answer is not in the context, say you don't know.\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}"
)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
question = "give me some info of product"
response = rag_chain.invoke(question)
response

'The product is the MacBook Air, as indicated in the title of the document. It includes features related to energy efficiency, compliance with ENERGY STAR guidelines, and disposal and recycling information. The device is designed to promote energy-efficient use and is shipped with power management enabled, causing it to sleep after 10 minutes of inactivity. It is important to dispose of the product and its battery separately from household waste at designated collection points to conserve resources and protect the environment. Additionally, the built-in battery should only be replaced or repaired by trained technicians to avoid risks such as overheating or fire.'

In [8]:
from typing import TypedDict, Annotated
from langgraph.types import interrupt, Command

In [9]:
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage


class ChatState(TypedDict):

     messages: str

In [10]:
def chat_node(state: ChatState):

    decision = interrupt({
        "type": "approval",
        "reason": "Model is about to answer a user question.",
        "question": state["messages"],
        "instruction": "Approve this question? yes/no"
    })
    
    if decision["approved"] == 'no':
        return {"messages": "Not approved."}

    else:
        response = llm.invoke(state["messages"])
        return {"messages": [response]}

In [11]:
from langchain_core.messages import AnyMessage, AIMessage

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

In [12]:
# 3. Build the graph: START -> chat -> END
builder = StateGraph(ChatState)

builder.add_node("chat", chat_node)

builder.add_edge(START, "chat")
builder.add_edge("chat", END)

# Checkpointer is required for interrupts
checkpointer = MemorySaver()

# Compile the app
app = builder.compile(checkpointer=checkpointer)


In [13]:
# Create a new thread id for this conversation
config = {"configurable": {"thread_id": '1234'}}

# ---- STEP 1: user asks a question ----
initial_input = {
    "messages":"Explain gradient descent in very simple terms."
}

# Invoke the graph for the first time
result = app.invoke(initial_input, config=config)

In [14]:
result

{'messages': 'Explain gradient descent in very simple terms.',
 '__interrupt__': [Interrupt(value={'type': 'approval', 'reason': 'Model is about to answer a user question.', 'question': 'Explain gradient descent in very simple terms.', 'instruction': 'Approve this question? yes/no'}, id='3f3904530a39f74eb775f20b0d1fc64e')]}

In [15]:
message = result['__interrupt__'][0].value
message

{'type': 'approval',
 'reason': 'Model is about to answer a user question.',
 'question': 'Explain gradient descent in very simple terms.',
 'instruction': 'Approve this question? yes/no'}

In [16]:
user_input = input(f"\nBackend message - {message} \n Approve this question? (y/n): ")

In [17]:
# Resume the graph with the approval decision
final_result = app.invoke(
    Command(resume={"approved": user_input}),
    config=config,
)

In [18]:
final_result

{'messages': 'Not approved.'}

In [19]:
QUERY_ROUTER_PROMPT = """
You are a Query Router Agent.
Your task is to read the user's query and reply with only one agent name based on the intent.

Rules:
1. If the user query is about math operations (calculation, equation, arithmetic, numbers) → reply "math"
2. If the user query is about getting information, explanation, facts, or knowledge → reply "knowledge"
3. If the user wants to book a ground / playground / turf / ground reservation → reply "ground"
4. For any other request, reply exactly: "out of my known"

Strict instructions:
- Reply with only one word.
- Do not answer the user query.
- Do not add punctuation or extra text.
- Allowed outputs only: math, knowledge, ground, out of my known

your query :-{query}
"""


In [20]:
from typing_extensions import Annotated
from operator import add
from langchain_core.tools import tool


In [65]:
from typing import TypedDict, List
import csv
import os
from datetime import datetime


class BookingSchema(TypedDict):
    user_id: str
    start_time: str
    end_time: str
    date: str
    status:str

CSV_FILE = "bookings.csv"


def get_booking(user_id: str) -> List[BookingSchema]:
    """
    Get all bookings for a specific user_id.
    
    Args:
        user_id: The user ID to filter bookings by
        
    Returns:
        List of booking dictionaries matching the user_id
    """
    bookings = read_bookings_from_csv()
    return [booking for booking in bookings if booking["user_id"] == user_id]


def is_time_slot_available(booking: BookingSchema) -> bool:
    """
    Check if a time slot is available (not conflicting with existing bookings).
    
    Args:
        booking: A booking dictionary with user_id, start_time, end_time, and date
        
    Returns:
        True if the time slot is available, False if it conflicts with existing bookings
    """
    existing_bookings = read_bookings_from_csv()
    
    # Parse the new booking times
    new_start = datetime.strptime(booking["start_time"], "%H:%M").time()
    new_end = datetime.strptime(booking["end_time"], "%H:%M").time()
    new_date = booking["date"]
    
    for existing_booking in existing_bookings:
        # Check if it's the same date
        if existing_booking["date"] == new_date:
            # Parse existing booking times
            existing_start = datetime.strptime(existing_booking["start_time"], "%H:%M").time()
            existing_end = datetime.strptime(existing_booking["end_time"], "%H:%M").time()
            
            # Check for time overlap
            # Two time slots overlap if:
            # - new_start < existing_end AND new_end > existing_start
            if new_start < existing_end and new_end > existing_start:
                return False
    
    return True


def save_booking_to_csv(booking: BookingSchema) -> str:
    """
    Save a booking to the CSV file.
    If the file doesn't exist, it will be created with headers.
    Checks for time slot conflicts before saving.
    
    Args:
        booking: A booking dictionary with user_id, start_time, end_time, and date
        
    Returns:
        Success message if booking is saved, or error message if time slot is occupied
    """
    required_fields = ["user_id", "start_time", "end_time", "date"]

    def is_complete(state: dict) -> bool:
        return all(
            field in state and state[field] not in ("", None)
            for field in required_fields
        )
    if is_complete(state=booking):
        print("inside the loop")
        # Check if time slot is available
        if not is_time_slot_available(booking):
            
            booking["status"]="Unable to book on this time. Please update your start time or end time."
            return booking
        file_exists = os.path.isfile(CSV_FILE)
        
        with open(CSV_FILE, 'a', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['user_id', 'start_time', 'end_time', 'date']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # Write header if file is new
            if not file_exists:
                writer.writeheader()
            
            # Write the booking data
            writer.writerow(booking)
            booking["status"]=f"your booking of ground done for {booking}"
    
    return booking


def read_bookings_from_csv() -> List[BookingSchema]:
    """
    Read all bookings from the CSV file.
    
    Returns:
        List of all booking dictionaries from the CSV file
    """
    bookings = []
    
    if not os.path.isfile(CSV_FILE):
        return bookings
    
    with open(CSV_FILE, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            booking: BookingSchema = {
                "user_id": row["user_id"],
                "start_time": row["start_time"],
                "end_time": row["end_time"],
                "date": row["date"]
            }
            bookings.append(booking)
    
    return bookings

ground_booking_bilder=StateGraph(BookingSchema)
ground_booking_bilder.add_node("save_booking_to_csv",save_booking_to_csv)
ground_booking_bilder.add_edge(START,"save_booking_to_csv")
ground_booking_bilder.add_edge("save_booking_to_csv",END)
checkpointer = MemorySaver()
ground_book_graph=ground_booking_bilder.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": '12345'}}


In [66]:
g_state={"status":""}

In [ ]:
from typing import TypedDict
import json
import openai

# -----------------------------
# BookingSchema definition
# -----------------------------
class BookingSchema(TypedDict, total=False):
    user_id: str
    start_time: str
    end_time: str
    date: str
    status: str

# -----------------------------
# OpenAI API key setup
# -----------------------------
OPENAI_API_KEY=""
openai.api_key = OPENAI_API_KEY

# -----------------------------
# Prompt template
# -----------------------------
PROMPT_TEMPLATE = PROMPT_TEMPLATE = """
You are a booking assistant.

Extract booking information from the user query.

Return ONLY a valid JSON object matching:
BookingSchema(TypedDict, total=False)
Allowed fields: user_id, start_time, end_time, date, status

STRICT RULES:
1. Output MUST be valid JSON only. No explanation.
2. Time format MUST be 24-hour HH:MM.
3. Convert AM/PM to 24-hour time.
4. Date format MUST be YYYY-MM-DD.
5. Include ONLY fields explicitly mentioned or clearly inferred.
6. If a field exists in the previous state and is NOT updated by the user,
   KEEP the previous value.
7. 🚫 If end_time is NOT mentioned, DO NOT include end_time.
8. 🚫 Never copy start_time into end_time.
9. Do NOT guess missing fields.
10. Do NOT include empty, null, or invalid values.

Previous state:
{previous_state}

User query:
"{user_query}"
"""



# -----------------------------
# Function to update single booking state
# -----------------------------
def update_booking_state(user_query: str, previous_state: BookingSchema) -> BookingSchema:
    prev_state_str = json.dumps(previous_state)
    prompt = PROMPT_TEMPLATE.format(user_query=user_query, previous_state=prev_state_str)

    # New syntax for v1.0+ OpenAI Python SDK
    response = openai.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": "You are a helpful booking assistant."},
            {"role": "user", "content": prompt}
        ],
       
    )

    ai_text = response.choices[0].message.content.strip()

    try:
        updated_state = json.loads(ai_text)
        
        return updated_state
    except json.JSONDecodeError:
        return previous_state



In [74]:
@tool
def math_tool(first_num: float, second_num: float, operation: str) -> dict:
    """
    Perform a basic arithmetic operation on two numbers.
    Supported operations: add, sub, mul, div
    """
    try:
        if operation == "add":
            result = first_num + second_num
        elif operation == "sub":
            result = first_num - second_num
        elif operation == "mul":
            result = first_num * second_num
        elif operation == "div":
            if second_num == 0:
                return {"error": "Division by zero is not allowed"}
            result = first_num / second_num
        else:
            return {"error": f"Unsupported operation '{operation}'"}
        
        return {"first_num": first_num, "second_num": second_num, "operation": operation, "result": result}
    except Exception as e:
        return {"error": str(e)}

class InitailStateState(TypedDict):
    query: str
    responce:str
    
def inital_chat(state:InitailStateState):
  
    query=state["query"]
    QUERY_ROUTER_PROMPT =f"""
    You are a Query Router Agent.
    Your task is to read the user's query and reply with only one agent name based on the intent.

    Rules:
    1. If the user query is about math operations (calculation, equation, arithmetic, numbers) → reply "math"
    2. If the user query is about getting information, explanation, facts, or knowledge → reply "knowledge"
    3. If the user wants to book a ground / playground / turf / ground reservation → reply "ground"
    4. For any other request, reply exactly: "out_of_my_known"

    Strict instructions:
    - Reply with only one word.
    - Do not answer the user query.
    - Do not add punctuation or extra text.
    - Allowed outputs only: math, knowledge, ground, out_of_my_known

    your query :- {query}
    """
    route=llm.invoke(QUERY_ROUTER_PROMPT).content.strip()

    return route
llm_with_tools = llm.bind_tools([math_tool])
# #dic=1
# def ground_book(state: InitailStateState):
    
#     if len(dic)!=5:
#         dic.append("ff")
#         decision = interrupt({
#             "type": "responce",
#             "question": "my ground book quation",
#             "instruction": "Approve this question? yes/no"
#         })
#     print("ds")
#     return state

def math(state:InitailStateState):

    query = state["query"]

    llm_with_tools = llm.bind_tools([math_tool])
    ai_msg = llm_with_tools.invoke(query)

    # Tool calling case
    if ai_msg.tool_calls:
        tool_call = ai_msg.tool_calls[0]

        tool_result = math_tool.invoke(tool_call["args"])

        return {
            "responce": str(tool_result["result"])
        }

    # Fallback
    return {
        "responce": ai_msg.content
    }

def knowledge(state:InitailStateState):
    query=state["query"]
    # Make the LLM tool-aware
    state["responce"]="dsa"
    return state

def out_of_my_known(state:InitailStateState):
    state["responce"]="out of know"
    return state


privious_state={
    "user_id": "",
    "start_time": "",    # 9:00 AM in 24-hour format
    "end_time": "",       # 5:30 PM in 24-hour format
    "date": ""
}
def ground(state:InitailStateState):
    query=state["query"]
    privious_state["user_id"]=123
    st=update_booking_state(query,privious_state)

    kt=ground_book_graph.invoke(st,config=config)
    
    privious_state["date"]=kt.get("date")
    privious_state["start_time"]=kt.get("start_time")
    privious_state["end_time"]=kt.get("end_time")
    
    if not kt.get("status"):
        
        human_answer = interrupt({
            "type":"query",
                "question": f"Please provide data for {privious_state}"
            })
    print(privious_state)
    state["responce"]=kt.get("status")
    return state


initail_graph=StateGraph(InitailStateState)
# initail_graph.add_node("inital_chat",inital_chat)
initail_graph.add_node("math",math)
initail_graph.add_node("knowledge",knowledge)
initail_graph.add_node("out_of_my_known",out_of_my_known)
initail_graph.add_node("ground",ground)

initail_graph.add_conditional_edges(START,inital_chat,{
        "math": "math",
        "knowledge": "knowledge",
        "out_of_my_known":"out_of_my_known",
        "ground":"ground"
    })
initail_graph.add_edge("math",END)
initail_graph.add_edge("knowledge",END)
initail_graph.add_edge("out_of_my_known",END)
initail_graph.add_edge("ground",END)
app=initail_graph.compile(checkpointer=checkpointer)


In [76]:
app.invoke({"query":"end time is 12"},config=config)

{'query': 'end time is 12', 'responce': 'out of know'}

In [232]:
app.invoke({"query":"i want to book ground"})

{'query': 'i want to book ground',
 '__interrupt__': [Interrupt(value={'question': 'Please provide equation'}, id='e95ae29667a8c49871d6c52b1d7d5446')]}

In [116]:
r["responce"].content

''

In [117]:
r=llm_with_tools.invoke("what is 3 +1")

In [118]:
r

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 73, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ee69c2ef48', 'id': 'chatcmpl-Co2JIKdZsNhcsSGRFZRis9pxxKaxv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b3043-e815-7b02-ac66-d9c0acfca6f7-0', tool_calls=[{'name': 'math_tool', 'args': {'first_num': 3, 'second_num': 1, 'operation': 'add'}, 'id': 'call_iO1pBKFTKRF1SBJHvB52WL3a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 73, 'output_tokens': 24, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
from typing import TypedDict
import json
import openai

# -----------------------------
# BookingSchema definition
# -----------------------------
class BookingSchema(TypedDict, total=False):
    user_id: str
    start_time: str
    end_time: str
    date: str
    status: str

# -----------------------------
# OpenAI API key setup
# -----------------------------
OPENAI_API_KEY=""
openai.api_key = OPENAI_API_KEY

# -----------------------------
# Prompt template
# -----------------------------
PROMPT_TEMPLATE = """
You are a booking assistant. Respond ONLY with a JSON object following this schema: 
BookingSchema(TypedDict, total=False) with fields user_id, start_time, end_time, date, status. 
Include ONLY the fields mentioned or inferred from the user query. 
If some fields were previously provided in the state, keep them and update with new info from this query.
Do NOT include extra text or explanation.

Previous state: {previous_state}

User query: "{user_query}"
"""

# -----------------------------
# Function to update single booking state
# -----------------------------
def update_booking_state(user_query: str, previous_state: BookingSchema) -> BookingSchema:
    prev_state_str = json.dumps(previous_state)
    prompt = PROMPT_TEMPLATE.format(user_query=user_query, previous_state=prev_state_str)

    # New syntax for v1.0+ OpenAI Python SDK
    response = openai.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": "You are a helpful booking assistant."},
            {"role": "user", "content": prompt}
        ],
       
    )

    ai_text = response.choices[0].message.content.strip()

    try:
        updated_state = json.loads(ai_text)
        
        return updated_state
    except json.JSONDecodeError:
        return previous_state

# -----------------------------
# Example usage
# -----------------------------
def k():
    state: BookingSchema = {}

    user_input1 = "Booking on 2025-12-20"
    state = update_booking_state(user_input1, state)
    print(state)

    # user_input2 = "Start at 10:00"
    # state = update_booking_state(user_input2, state)
    # print(state)

    # user_input3 = "User ID is U123"
    # state = update_booking_state(user_input3, state)
    # print(state)

k()


{'date': '2025-12-20'}
{'date': '2025-12-20'}


In [ ]:


from langgraph.graph.message import add_messages

class ChatState(TypedDict):

    query: str
    responce:str

In [225]:
def chat_node(state: ChatState):

    decision = interrupt({
        "type": "responce",
        "question": "my ground book quation",
        "instruction": "Approve this question? yes/no"
    })
    
    if decision["responce"] == 'no':
        return {"messages": [AIMessage(content="Not approved.")]}

    else:
        response = llm.invoke(state["messages"])
        return {"messages": [response]}

In [226]:
# 3. Build the graph: START -> chat -> END
builder = StateGraph(ChatState)

builder.add_node("chat", chat_node)

builder.add_edge(START, "chat")
builder.add_edge("chat", END)

# Checkpointer is required for interrupts
checkpointer = MemorySaver()

# Compile the app
app = builder.compile(checkpointer=checkpointer)

In [227]:
# Create a new thread id for this conversation
config = {"configurable": {"thread_id": '1234'}}

# ---- STEP 1: user asks a question ----
initial_input = {
    "messages": [
        ("user", "Explain gradient descent in very simple terms.")
    ]
}

# Invoke the graph for the first time
result = app.invoke(initial_input, config=config)

In [228]:
message = result['__interrupt__'][0].value
message

{'type': 'responce',
 'question': 'my ground book quation',
 'instruction': 'Approve this question? yes/no'}

In [229]:
# Resume the graph with the approval decision
final_result = app.invoke(
    Command(resume={"responce": "user_input"}),
    config=config,
)

In [230]:
final_result

{'messages': [HumanMessage(content='Explain gradient descent in very simple terms.', additional_kwargs={}, response_metadata={}, id='879db7c7-1bd4-47ca-bdc4-20dd0c7d8105'),
  AIMessage(content='Sure! Imagine you\'re on a hill and your goal is to find the lowest point in the valley. You can\'t see the whole valley, but you can feel the slope of the ground beneath your feet.\n\n1. **Start at a Point**: You begin at a random spot on the hill.\n2. **Feel the Slope**: You check which direction is downhill (the steepest slope).\n3. **Take a Step**: You take a small step in that downhill direction.\n4. **Repeat**: You keep checking the slope and taking steps until you can’t go any lower.\n\nIn the context of machine learning, gradient descent is a method used to minimize a function (like finding the best fit for a line in data). The "hill" represents the error or loss, and the "lowest point" is where the error is minimized. By repeatedly adjusting the parameters (like taking steps), we find t